# Imports, global settings, and definitions of aggregate and helper functions

In [ ]:
import json
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("pastel")

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 11pt font in plots, to match 11pt font in document
    "axes.labelsize": 11,
    "font.size": 11,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 9,
    "legend.title_fontsize": 9,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "text.latex.preamble": "\\usepackage{latexsym}\\usepackage{amsmath}"
}

plt.rcParams.update(tex_fonts)
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
def checkData(data):
    runs = {}
    algo = {}
    length = {}
    
    for run in data:
        seed = run['seed']
        size = run['numNodes']
            
        runId= run['run']
            
        if seed not in runs:
            runs[seed] = {}
            algo[seed] = {}
        if size not in runs[seed]:
            runs[seed][size] = {}
            algo[seed][size] = {}
        if runId not in runs[seed][size]:
            runs[seed][size][runId] = run['paths']
            algo[seed][size][runId] = run['algoName']
        
        # check for increasing length
        lastLength = 0
        for i, path in enumerate(run['paths']):
            info = f"algo: {run['algoName']} vs {algo[seed][size][runId]}, seed: {seed}, graph: {size}, run: {runId}, path: {i}, length: {path['length']} vs {runs[seed][size][runId][i]['length']} ({runs[seed][size][runId][i]['length'] - runs[seed][size][runId][0]['length']})"
            
            if len(path['path']) != len(set(path['path'])):
                print(path['path'], "path is not loopless", info)
                print('Data is invalid')
                return
            
            if path['length'] < lastLength:
                print(path['length'], '<', lastLength, info)
                print('Data is invalid')
                return
            
            lastLength = path['length'];
            
            if runs[seed][size][runId][i]['path'][0] != path['path'][0]:
                print(f"Different source node.. s = {runs[seed][size][runId][i]['path'][0]} vs {path['path'][0]}", info)
                print('Data is invalid')
                return
            
            if runs[seed][size][runId][i]['path'] != path['path']:
                print(runs[seed][size][runId][i]['path'],'!=', path['path'], 'different paths', info)
                print('Data is invalid')
                return
            
    print('Data looks valid')
        

def toGraphExponent(numNodes):
    return math.ceil(math.log2(numNodes))

def getListOfSSSPSkipK(exploredNodesAll):
    result = []
    for i, exploredNodesI in enumerate(exploredNodesAll):
        for exploredNodes in exploredNodesI:
            if exploredNodes == 0:
                result.append(i + 1)
                
    return result
                
def getListOfSSSPSkipDevIds(exploredNodesAll, paths):
    result = []
    for j, exploredNodesI in enumerate(exploredNodesAll):
        pathHops = len(paths[j]['path'])
        size = len(exploredNodesI) + 1     # no deviations from the last node
        assert(pathHops >= size)
        for i, exploredNodes in enumerate(exploredNodesI):
            if exploredNodes == 0:
                result.append((pathHops - size + i) / pathHops)
    
    return result

def getYellowGraphSizes(stats):
    yellowGraphSizes = []
    
    for stat in stats:
        if stat['algo'] != 'feng':
            continue
        
        numNodes = stat['numNodes']
        graph = toGraphExponent(numNodes)
        for k, ygs in enumerate(stat['stats']['yellowGraphSizes']):
            pathLen = len(stat['paths'][k]['path'])
            numYellowGraphs = len(ygs)
            for i , y in enumerate(ygs):
                if y == 0:
                    continue
                    
                yellowGraphSizes.append({
                    'graph': graph,
                    'graphName': f'$n=2^{{{graph}}}$',
                    'k': k + 1,
                    'devId': pathLen - numYellowGraphs + i,
                    'devIdRel': (pathLen - numYellowGraphs + i) / pathLen,
                    'yellowGraphSize': y,
                    'yellowGraphSizeLog': np.log2(y),
                    'yellowGraphSizeRel': y / numNodes
                })
                
    return yellowGraphSizes

def getExploredNodes(stats):
    exploredNodes = []
    
    skippedComputations = [[[] for j in range(0,50)] for i in range(0, 21)]
    for run in stats:
        # if run['algoName'] != 'Feng':
            # continue
        
        for k, elist in enumerate(run['stats']['exploredNodes']):
            for e in elist:
                skippedComputations[run['run']][k].append(e == 0)
    
    
    for stat in stats:
        numNodes = stat['numNodes']
        graph = toGraphExponent(numNodes)
        guided = stat['guided']
        algoName = stat['algoName']
        for k, exn in enumerate(stat['stats']['exploredNodes']):
            pathLen = len(stat['paths'][k]['path'])
            numDeviations = len(exn)
            for i , expNodes in enumerate(exn):
                if expNodes == 0:
                    continue
                
                exploredNodes.append({
                    'algoName': algoName,
                    'graph': graph,
                    'graphName': f'$2^{{{graph}}}$',
                    'guided': guided,
                    'skipped': skippedComputations[stat['run']][k][i],
                    'k': k + 1,
                    'devId': pathLen - numDeviations + i,
                    'devIdRel': (pathLen - numDeviations + i) / pathLen,
                    'exploredNodes': expNodes,
                    'exploredNodesLog': np.log2(expNodes),
                    'exploredNodesRelTotal': expNodes / numNodes,
                    'exploredNodesRelYellow': expNodes / (stat['stats']['yellowGraphSizes'][k][i] if 'yellowGraphSizes' in stat['stats'] else numNodes)
                })
                
    return exploredNodes

def accumulateSkipDataPerRun(stats):
    testDataPerRunRaw = []
    
    for j, run in enumerate(stats):
        numNodes = run['graph']['numNodes'] if 'graph' in run else run['numNodes']
        graphNodes = toGraphExponent(numNodes)
        skippedK = getListOfSSSPSkipK(run['stats']['exploredNodes'])

        skippedTotal = run['stats']['SSCShortestDeviation'] + run['stats']['SSCSecondShortestDeviation'] + run['stats']['SSCShortestDeviationLength'] + run['stats']['SSCSecondShortestDeviationLength']

        ssspNotSkipedByFirstDeviation = run['stats']['totalPathComputations'] - run['stats']['SSCShortestDeviation'] - run['stats']['SSCShortestDeviationLength']
        
        testDataPerRunRaw.append({
            'graph': graphNodes,
            'directness': run['directness'],
            'n': f'$2^{{{graphNodes}}}$',
            'guided': run['guided'],
            'algo': run['algo'],
            'algoName': run['algoName'],
            'numNodes': numNodes,
            'path-computations-total': run['stats']['totalPathComputations'],
            'stopped-early': run['stats']['ssspStoppedEarly'],
            'stopped-early-rel-total':  run['stats']['ssspStoppedEarly'] / run['stats']['totalPathComputations'],
            'stopped-early-rel-non-skipped':  0 if (run['stats']['totalPathComputations'] - skippedTotal) == 0 else run['stats']['ssspStoppedEarly'] / (run['stats']['totalPathComputations'] - skippedTotal),
            'skipped-total': skippedTotal,
            'skipped-total-rel': skippedTotal / run['stats']['totalPathComputations'],
            'skipped-sd': run['stats']['SSCShortestDeviation'],
            'skipped-ssd': run['stats']['SSCSecondShortestDeviation'],
            'skipped-sdl': run['stats']['SSCShortestDeviationLength'],
            'skipped-ssdl': run['stats']['SSCSecondShortestDeviationLength'],
            'skipped-sd-rel': run['stats']['SSCShortestDeviation'] / run['stats']['totalPathComputations'],
            'skipped-ssd-rel': run['stats']['SSCSecondShortestDeviation'] / run['stats']['totalPathComputations'],
            'skipped-ssd-rel-2': (run['stats']['SSCSecondShortestDeviation'] / ssspNotSkipedByFirstDeviation) if ssspNotSkipedByFirstDeviation > 0 else 0,
            'skipped-sdl-rel': run['stats']['SSCShortestDeviationLength'] / run['stats']['totalPathComputations'],
            'skipped-ssdl-rel': run['stats']['SSCSecondShortestDeviationLength'] / run['stats']['totalPathComputations'],   
            'skipped-ssdl-rel-2': (run['stats']['SSCSecondShortestDeviationLength'] / ssspNotSkipedByFirstDeviation) if ssspNotSkipedByFirstDeviation > 0 else 0,        
        })
        
    return pd.DataFrame(testDataPerRunRaw)
    
def accumulateSkipDataPerDev(stats):
    testDataPerDevRaw = []
    
    for j, run in enumerate(stats):
        numNodes = run['graph']['numNodes'] if 'graph' in run else run['numNodes']
        graphNodes = toGraphExponent(numNodes)
        skippedK = getListOfSSSPSkipK(run['stats']['exploredNodes'])
        skippedDevId = getListOfSSSPSkipDevIds(run['stats']['exploredNodes'], run['paths'])

        assert(len(skippedK) == len(skippedDevId))

        for i in range(0,len(skippedK)):
            testDataPerDevRaw.append({
                'graph': graphNodes,
                'n': f'$2^{{{graphNodes}}}$',
                'guided': run['guided'],
                'algo': run['algo'],
                'algoName': run['algoName'],
                'numNodes': numNodes,
                'skipped-k': skippedK[i],
                'skipped-dev-id': skippedDevId[i]
            })
            
    return pd.DataFrame(testDataPerDevRaw)

def accumulatePathData(stats):
    pathDataRaw = []
    
    for j, run in enumerate(stats):
        numNodes = run['graph']['numNodes'] if 'graph' in run else run['numNodes']
        graphNodes = toGraphExponent(numNodes)

        for k, path in enumerate(run['paths']):
            pathDataRaw.append({
                'algo': run['algo'],
                'algoName': run['algoName'],
                'graph': graphNodes,
                'n': f'$2^{{{graphNodes}}}$',
                'guided': run['guided'],
                'algo': run['algo'],
                'k': k+1,
                'length': path['length'] - run['paths'][0]['length'],
                'hops': len(path['path']),
                'devId': path['deviationNodeIndex'],
                'parentPathId': path['parentPathId']
            })
               
    return pd.DataFrame(pathDataRaw)

def getAlgoName(stats):
    baseName = stats['algo'].capitalize()
    if stats['guided'] or stats['attemptSSSPSkips'] > 0:
        baseName +=  '-'
        
    if stats['guided']:
        baseName += 'g'
        
    if stats['attemptSSSPSkips'] > 0:
        baseName += 's'
        
    if stats['attemptSSSPSkips'] == 2:
        baseName += '2'
        
    if 'skipByLength' in stats and stats['skipByLength']:
        baseName += '-l'
    
    return baseName

def setLegendAlgoFont(axis):
    handles, labels = axis.get_legend_handles_labels()
    newlabels = ['\\textsc{' + l + '}' for l in labels]
    axis.legend(handles,newlabels)
    
def finalizePlot(fileName=None):
    plt.tight_layout()
    if fileName is not None:
        plt.savefig(fileName, bbox_inches='tight')
    plt.show()

# Load data and show some metadata to make sure that everything is there

JSON sructure:
```
[{algo, guided, attemptSSSPSkips, numNodes, numEdges, seed, run,
  paths: [{path, length, deviationNodeIndex, parentPathId}], 
  stats: {SSCShortestDeviation, SSCSecondShortestDeviation, SSCShortestDeviationLength, SSCSecondShortestDeviationLength, ssspStoppedEarly, totalPathComputations, 
  yellowGraphSizes:[][], numExpressEdges:[][], exploredNodes:[][]}
}]
```


In [ ]:
def loadStatistics(fileNames):
    data = ""
    
    for i in fileNames:
        file = f'../data/statistics/stats.{i}.json'
        print(f'Loading file "{file}"')
        with open(file, 'r') as file:
            data = data + file.read().replace('\n', '')

    print('Done.')      
    print('Parse JSON')

    stats = json.loads("[" + data[:-1] + "]")  # remove closing comma and make it an array of objects

    print(f'Done. {len(stats)} rows.')
    print('Add Data')

    for i in range(0, len(stats)):
        numNodes = stats[i]['numNodes']
        run = stats[i]['run']

        graphExp = toGraphExponent(numNodes)
        stats[i]['algoName'] = getAlgoName(stats[i])
        stats[i]['directness'] = 1.0

        stats[i]['graphExp'] = graphExp
        stats[i]['graphName'] = 2**(graphExp-20)
        stats[i]['run'] = run + 1
        stats[i]['count'] = 1

    print('Done.')
    print('Check Data')

    checkData(stats)

    return stats


statsGnp4 = loadStatistics(['feng-gs2.gnp4','yen-gs2.gnp4','yen-s2.gnp4'])
statsGnp4s1 = loadStatistics(['feng-gs1.gnp4','yen-gs1.gnp4','yen-s1.gnp4'])
statsGnp64 = loadStatistics(['feng-gs2.gnp64','yen-gs2.gnp64','yen-s2.gnp64'])
statsGnp64s1 = loadStatistics(['feng-gs1.gnp64','yen-gs1.gnp64','yen-s1.gnp64'])
statsGridsAll = loadStatistics(['grids'])
statsGrids = [x for x in statsGridsAll if x['attemptSSSPSkips'] == 2 and x['graphExp'] <= 26]
statsGridss1 = [x for x in statsGridsAll if x['attemptSSSPSkips'] == 1 and x['graphExp'] <= 24]

# for run in statsGnp4:
#     if run['algo'] == 'yen' and run['graphName'] == 2:
#         print(run['algoName'],
#               run['stats']['SSCShortestDeviation'], 
#               run['stats']['SSCShortestDeviationLength'], 
#               run['stats']['SSCSecondShortestDeviation'], 
#               run['stats']['SSCSecondShortestDeviationLength'], run['paths'][0]['path'][0])

df = pd.DataFrame(statsGridss1)

# print(df[(df['graphName'] == 2) & (df['algo'] == 'yen')] \
# .filter(items=['algoName', 'graphName', 'stats'])) \
# .groupby(by=["algo", 'guided', 'graphName', 'run']) \
# .sum()

df \
.filter(items=['algoName', 'guided', 'graphName', 'run', 'count']) \
.groupby(by=["algoName", 'graphName']) \
.sum()

In [ ]:
directnessData = []

for i in range(0,11):
    dn = str(i / 10.0)
    file = f'../data/statistics/directness/gnp64-4-{dn}.json'
    print(f'Loading file "{file}"')
    with open(file, 'r') as file:
        tmp = json.loads("[" + file.read()[:-2] + "]")    # remove closing comma and new line and make it an array of objects
        for row in tmp:
            row['directness'] = i / 10.0
            row['dn'] = dn
            
            numNodes = row['numNodes']
            run = row['run']

            graphExp = toGraphExponent(numNodes)
            row['algoName'] = getAlgoName(row)

            row['graphExp'] = graphExp
            row['graphName'] = 2**(graphExp-20)
            row['run'] = run + 1
            row['count'] = 1
            
        directnessData = directnessData + tmp
        # directnessData = pd.concat([directnessData, pd.DataFrame(tmp)])

# for run in statsGnp4:
    # if run['algoName'] == 'Yen-gs2' and run['graphName'] == 64:
    #     directnessData.append(run)
        
print('Done.')

pd.DataFrame(directnessData) \
.filter(items=['algoName', 'guided', 'graphName', 'run', 'count', 'directness']) \
.groupby(by=["algoName", 'graphName', 'directness']) \
.sum()

# Accumulate Data stoppedEarlyDataGrids

In [ ]:
testDataPerRun4 = accumulateSkipDataPerRun(statsGnp4)
testDataPerRun64 = accumulateSkipDataPerRun(statsGnp64)
testDataPerRunGrids = accumulateSkipDataPerRun(statsGrids)

testDataPerDev4 = accumulateSkipDataPerDev(statsGnp4)
testDataPerDev64 = accumulateSkipDataPerDev(statsGnp64)
testDataPerDevGrids = accumulateSkipDataPerDev(statsGrids)

pathData4 = accumulatePathData(statsGnp4)
pathData64 = accumulatePathData(statsGnp64)
pathDataGrids = accumulatePathData(statsGrids)

directnesSkipData = accumulateSkipDataPerRun(directnessData)

yellowGraphData4 = pd.DataFrame(getYellowGraphSizes(statsGnp4))
yellowGraphData64 = pd.DataFrame(getYellowGraphSizes(statsGnp64))
yellowGraphDataGrids = pd.DataFrame(getYellowGraphSizes(statsGrids))

exploredNodesData4 = pd.DataFrame(getExploredNodes(statsGnp4s1))
exploredNodesData64 = pd.DataFrame(getExploredNodes(statsGnp64s1))
exploredNodesDataGrids = pd.DataFrame(getExploredNodes(statsGridss1))

stoppedEarlyData4 = pd.DataFrame(accumulateSkipDataPerRun(statsGnp4s1))
stoppedEarlyData64 = pd.DataFrame(accumulateSkipDataPerRun(statsGnp64s1))
stoppedEarlyDataGrids = pd.DataFrame(accumulateSkipDataPerRun(statsGridss1))

testDataPerRun4

# Polts about Skipping SSSP

In [ ]:
fengAlgo = 'Feng-gs2-l'
yenAlgo = 'Yen-gs2-l'


fig, (a0, a1, a2) = plt.subplots(1, 3, sharey=True, gridspec_kw={'width_ratios': [9, 7, 4]}, figsize=(10, 4.5))

sns.boxplot(data=testDataPerRun4[testDataPerRun4['algoName'] != fengAlgo], x='n', y='skipped-total-rel', hue='guided', 
            order=[f'$2^{{{i}}}$' for i in range(20, 29)], ax=a0)
a0.set(xlabel='$n$ of $\mathcal{G}(n,\\frac{4}{n})$ graph', ylabel='skipped ratio' #, title="Ratio of SSSP computations skipped (total)"
      )

sns.boxplot(data=testDataPerRun64[testDataPerRun64['algoName'] != fengAlgo], x='n', y='skipped-total-rel', hue='guided', 
            order=[f'$2^{{{i}}}$' for i in range(20, 27)], ax=a1)
a1.set(xlabel='$n$ of $\mathcal{G}(n,\\frac{64}{n})$ graph'#, ylabel='skipped ratio', title="Ratio of SSSP computations skipped (total)"
      )

sns.boxplot(data=testDataPerRunGrids[testDataPerRunGrids['algoName'] != fengAlgo], x='n', y='skipped-total-rel', hue='guided', 
            order=[f'$2^{{{i}}}$' for i in range(20, 27, 2)], ax=a2)
a2.set(xlabel='$n$ of $\\operatorname{Grid}(n,4,0.8)$ graph'#, ylabel='skipped ratio', title="Ratio of SSSP computations skipped (total)"
      )

a1.set_ylabel('')
a2.set_ylabel('')
a0.get_legend().remove()
a2.get_legend().remove()
plt.suptitle("Ratio of SSSP computations skipped", fontsize=16)

plt.tight_layout()
plt.savefig(f'plots/sssp-skipped-rel/total-gnp.pdf')
plt.show()

for graph in ['gnp4', 'gnp64', 'grid']:
    if graph == 'gnp4' :
        dataRun = testDataPerRun4
        dataDev = testDataPerDev4
        maxNumNodeLog = 29
        xlabel='$n$ of $\mathcal{G}(n,\\frac{4}{n})$ graph'
        graphOrder = [f'$2^{{{i}}}$' for i in range(20, maxNumNodeLog)]
    elif graph == 'gnp64':
        dataRun = testDataPerRun64
        dataDev = testDataPerDev64   
        maxNumNodeLog = 27
        xlabel='$n$ of $\mathcal{G}(n,\\frac{64}{n})$ graph'
        graphOrder = [f'$2^{{{i}}}$' for i in range(20, maxNumNodeLog)]
    elif graph == 'grid':
        dataRun = testDataPerRunGrids
        dataDev = testDataPerDevGrids 
        maxNumNodeLog = 27
        xlabel='$n$ of $\\operatorname{Grid}(n,4,0.8)$ graph'
        graphOrder = [f'$2^{{{i}}}$' for i in range(20, maxNumNodeLog, 2)]
        
    ax = sns.boxplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='path-computations-total', color=sns.color_palette('pastel', 1)[0], order=graphOrder)
    ax.set(xlabel=xlabel, ylabel='num paths', title="Total Paths Computed")
    plt.tight_layout()
    # plt.savefig(f'plots/paths-computed-total-{graph}.pdf')
    plt.show()

    # ax = sns.boxplot(data=testDataPerRun[testDataPerRun['algoName'] != yenAlgo], x='n', y='skipped-total', hue='guided', order=graphOrder)
    # ax.set(xlabel='$n$ of $G(n,\\frac{4}{n})$ graph', ylabel='num computations skipped', title="Total of SSSP computations skipped")
    # plt.savefig('plots/skipped-total.pdf')
    # plt.show()

#     ax = sns.boxplot(data=dataRun[dataRun['algoName'] != fengAlgo], x='n', y='skipped-total-rel', hue='guided', order=graphOrder)
#     ax.set(xlabel=xlabel, ylabel='skipped ratio', title="Ratio of SSSP computations skipped (total)")
#     plt.savefig(f'plots/sssp-skipped-rel/total-{graph}.pdf')
#     plt.show()

    fig, ((a0, a1), (a2, a3)) = plt.subplots(2, 2, sharex=True, gridspec_kw={'width_ratios': [1, 1]}, figsize=(10, 6))
    
    sns.boxplot(data=dataRun[dataRun['algoName'] != fengAlgo], x='n', y='skipped-sd-rel', hue='guided', order=graphOrder, ax=a0)
    a0.set(ylabel='skipped ratio', title="\dots shortest deviation (SD)")

    sns.boxplot(data=dataRun[dataRun['algoName'] != fengAlgo], x='n', y='skipped-sdl-rel', hue='guided', order=graphOrder, ax=a1)
    a1.set(title="\dots shortest deviation length (SDL)")

    sns.boxplot(data=dataRun[dataRun['algoName'] != fengAlgo], x='n', y='skipped-ssd-rel', hue='guided', order=graphOrder, ax=a2)
    a2.set(xlabel=xlabel, ylabel='skipped ratio', title="\dots second shortest deviation (SSD)")

    sns.boxplot(data=dataRun[dataRun['algoName'] != fengAlgo], x='n', y='skipped-ssdl-rel', hue='guided', order=graphOrder, ax=a3)
    a3.set(xlabel=xlabel, title="\dots second shortest deviation length (SSDL)")
    
    a0.set_xlabel('')
    a1.set_xlabel('')
    a1.set_ylabel('')
    a3.set_ylabel('')
    a0.get_legend().remove()
    a1.get_legend().remove()
    a2.get_legend().remove()
    plt.suptitle("Ratio of SSSP computations skipped by$\dots$", fontsize=16)
    plt.tight_layout(w_pad=1.0, h_pad=0.5)
    plt.savefig(f'plots/sssp-skipped-rel/combined-{graph}.pdf')
    plt.show()

    ax = sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-total-rel', label='total')
    sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-sd-rel', label='sd')
    sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-sdl-rel', label='sdl')
    plt.legend(title='Skip Type', ncol=3)
    plt.tight_layout()
    # plt.savefig(f'plots/skipped-total-rel-3.1-{graph}.pdf')
    plt.show()

    ax = sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-ssd-rel', label='ssd')
    sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-sdl-rel', label='sdl')
    sns.lineplot(data=dataRun[dataRun['algoName'] == yenAlgo], x='n', y='skipped-ssdl-rel', label='ssdl')
    ax.set(xlabel=xlabel, ylabel='skipped ratio', title="Ratio of SSSP computations skipped")
    plt.legend(title='Skip Type', ncol=3)
    plt.tight_layout()
    # plt.savefig(f'plots/skipped-total-rel-3.2-{graph}.pdf')
    plt.show()

    ax = sns.violinplot(data=dataDev[dataDev['algoName'] != fengAlgo], x='n', y='skipped-k', hue='guided', order=graphOrder)
    ax.set(xlabel=xlabel, ylabel='k', 
           title="Distribution of skipped SSSP computations to $k^{\mathrm{th}}$ shortest path")
    plt.tight_layout()
    # plt.savefig(f'plots/skipped-k-{graph}.pdf')
    plt.show()
    
    # ax = sns.boxplot(data=dataDev, x='n', hue='guided', y='skipped-dev-id', palette="pastel", order=graphOrder)
    # ax.set(xlabel=xlabel, ylabel='relative deviation id', title="Relative deviation id where SSSP was skipped")
    # plt.tight_layout()
    # plt.savefig(f'plots/skipped-dev-id-{graph}.pdf')
    # plt.show()

    ax = sns.violinplot(data=dataDev, x='n', hue='guided', y='skipped-dev-id', palette="pastel", order=graphOrder)
    ax.set(xlabel=xlabel, ylabel='relative deviation id', title="Relative deviation id where SSSP was skipped")
    plt.tight_layout()
    plt.show()

    # ax = sns.lineplot(data=dataDev, x='graph', hue='guided', y='skipped-dev-id', palette="pastel")
    # ax.set(xlabel='$\log_2$ of ' + xlabel, ylabel='relative deviation id', title="Relative deviation id where SSSP was skipped")
    # plt.tight_layout()
    # plt.show()

In [ ]:
ax = sns.lineplot(data=directnesSkipData, x='directness', y='skipped-total-rel', label='total')
sns.lineplot(data=directnesSkipData, x='directness', y='skipped-sd-rel', label='SD')
sns.lineplot(data=directnesSkipData, x='directness', y='skipped-sdl-rel', label='SDL')
sns.lineplot(data=directnesSkipData, x='directness', y='skipped-ssd-rel', label='SSD')
sns.lineplot(data=directnesSkipData, x='directness', y='skipped-ssdl-rel', label='SSDL')
plt.legend(title='Skip Type', ncol=2)
ax.set(xlabel='$q$ of $q$-directed $\mathcal{G}(n,\\frac{4}{n})$ graphs with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
plt.tight_layout()
plt.savefig('plots/sssp-skipped-rel/directness-gnp4.pdf')
plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-sd-rel', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-sdl-rel', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-ssd-rel', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-ssdl-rel', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-ssd-rel-2', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# ax = sns.boxplot(data=directnesSkipData, x='directness', y='skipped-ssdl-rel-2', color=sns.color_palette('pastel', 1)[0])
# ax.set(xlabel='directness of $\mathcal{G}(n,\\frac{4}{n})$ graph with $n=2^{26}$', ylabel='ratio', title="Ratio of skipped SSSP computations")
# plt.show()

# Yellow Graph Sizes

In [ ]:
for deg in [4, 64]:
    if deg == 4 :
        dataYG = yellowGraphData4
    elif deg == 64:
        dataYG = yellowGraphData64 
        
    maxNumNodeLog = 29 if deg == 4 else 27

    fig, (a0, a1) = plt.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios': [1, 1]}, figsize=(10, 5))
    
    sns.lineplot(data=dataYG, x='devIdRel', y='yellowGraphSizeLog', hue='graph', palette='crest', ci=None, ax=a0)
    a0.set(xlabel='Deviation node index relative between $s$ and $t$', ylabel='Size of yellow graph ($\log_2$)',
           title=f"mean")
    a0.set(xlim=(0, 1))
    plt.yticks(np.arange(0, 29, 2.0))
    
    a0.get_legend().remove()
    
    # plt.legend(title='Graphs', ncol=3, labels=[f'$n=2^{{{g}}}$' for g in range(20, maxNumNodeLog)])
    
    # plt.savefig(f'plots/yellow-graph-sizes/gnp{deg}-mean.pdf')
    # plt.show()

    # ax = sns.scatterplot(data=dataYG, x='devIdRel', y='yellowGraphSizeLog', hue='graph', palette='crest', ci=None)
    # ax.set(xlabel='Deviation node index relative between $s$ and $t$', ylabel='Size of yellow graph ($\log_2$)',
    #        title=f"Yellow graph sizes (mean) for $\mathcal{{G}}(n,p)$ graphs with $p=\\frac{{{deg}}}{{n}}$")
    # ax.set(xlim=(0, 1))
    # plt.legend(title='Graphs', ncol=3, labels=[f'$n=2^{{{g}}}$' for g in range(20,maxNumNodeLog)])
    # plt.yticks(np.arange(0, 29, 2.0))
    # plt.savefig(f'plots/yellow-graph-sizes/gnp{deg}-raw.pdf')
    # plt.show()

    paletteBackup = sns.color_palette()
    
    sns.set_palette('crest', 9)
    for i, g in enumerate(range(20, maxNumNodeLog)):
        sns.regplot(data=dataYG[dataYG['graph']==g], x='devIdRel', y='yellowGraphSizeLog', scatter=False, label=f'$n=2^{{{g}}}$', color=sns.color_palette('crest', 9)[i], ax=a1)

    a1.set(xlabel='Deviation node index relative between $s$ and $t$', ylabel='Size of yellow graph ($\log_2$)',
           title=f"regression")
    a1.set(xlim=(0, 1))
    plt.legend(ncol=3)
    # plt.legend(title='Graphs', ncol=3)
    # plt.yticks(np.arange(0, 29, 2.0))
    # plt.savefig(f'plots/yellow-graph-sizes/gnp{deg}-regression.pdf')
    a1.set_ylabel('')
    
    
    plt.suptitle(f"Yellow graph sizes for $\mathcal{{G}}(n,p)$ graphs with $p=\\frac{{{deg}}}{{n}}$", fontsize=16)
    
    plt.tight_layout()
    plt.savefig(f'plots/yellow-graph-sizes/gnp{deg}.pdf')
    plt.show()
    
    sns.set_palette(paletteBackup)

# Path Lengths

In [ ]:
for graph in ['gnp4', 'gnp64', 'grids']:
    
    maxNumNodeLog = 29 if graph == 'gnp4' else 27
    
    if graph == 'gnp4':
        dataP = pathData4
        legendTitle = '$\\mathcal{G}(n,p)$ Graphs with $p=\\frac{4}{n}$'
        graphList = range(20, maxNumNodeLog)
    elif graph == 'gnp64':
        dataP = pathData64
        legendTitle = '$\\mathcal{G}(n,p)$ Graphs with $p=\\frac{64}{n}$'
        graphList = range(20, maxNumNodeLog)
    elif graph == 'grids':
        dataP = pathDataGrids
        legendTitle = '$\\mathcal{G}(n,r,p)$ Graphs with $r=4$ and $p=0.8$'
        graphList = range(18, maxNumNodeLog, 2)
    
    ax = sns.lineplot(data=dataP[dataP['algoName'] == 'Yen-gs2'], x='k', y='length', hue='graph', legend='full', palette='crest', ci=None)
    ax.set(title='Length of the $k^{\\text{th}}$ shortest path')
    plt.legend(title=legendTitle, ncol=3, labels=[f'$n=2^{{{g}}}$' for g in graphList])
    finalizePlot(f'plots/paths/path-length-{graph}.pdf')

    ax = sns.lineplot(data=dataP[dataP['algoName'] == 'Yen-gs2'], x='k', y='hops', hue='graph', legend='full', palette='crest', ci=None)
    ax.set(title='Number of hops of the $k^{\\text{th}}$ shortest path')
    plt.legend(title=legendTitle, ncol=3, labels=[f'$n=2^{{{g}}}$' for g in graphList])
    if graph == 'grids':
        plt.yscale('log', base=2)
        
    finalizePlot(f'plots/paths/num-hobs-{graph}.pdf')

    paletteBackup = sns.color_palette()
    sns.set_palette('crest', len(graphList))
    for g in graphList:
        ax = sns.regplot(data=dataP[(dataP['algoName'] == 'Yen-gs2') & (dataP['graph'] == g)], x='k', y='hops', scatter=False, label=f'$n=2^{{{g}}}$')

    ax.set(title='Number of hops of the $k^{\\text{th}}$ shortest path (regression)')
    
    if graph == 'grids':
        plt.yscale('log', base=2)
    
    plt.legend(title=legendTitle, ncol=3)
    finalizePlot(f'plots/paths/num-hobs-regression-{graph}.pdf')
    sns.set_palette(paletteBackup)

# Explored Nodes

In [ ]:
fig, a = plt.subplots(3, 2, sharey=True, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10, 9))


for j, graph in enumerate(['gnp4','gnp64','grids']):
    if graph == 'gnp4':
        dataE = exploredNodesData4
        graphList = range(20, 29)
        xAxisLabel = '$n$ of $\mathcal{G}(n,\\frac{4}{n})$ graph'
    elif graph == 'gnp64':
        dataE = exploredNodesData64
        graphList = range(20, 27)
        xAxisLabel = '$n$ of $\mathcal{G}(n,\\frac{64}{n})$ graph'
    elif graph == 'grids':
        dataE = exploredNodesDataGrids
        graphList = range(18, 25, 2)
        xAxisLabel = '$n$ of $\operatorname{Grid}(n,4,0.8)$ graph'
    
    algoOrder = ['Feng-gs', 'Yen-gs', 'Yen-s']
    graphOrder = [f'$2^{{{i}}}$' for i in graphList]
    
    # fig, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10, 5))
    
    sns.boxplot(data=dataE[(dataE['skipped']==False)], x='graphName', y='exploredNodesRelTotal', hue='algoName', hue_order=algoOrder, order=graphOrder, ax=a[j][0])
    
    a[j][0].set_yscale('log', base=2)
    if j == 0:
        a[j][0].set(xlabel=xAxisLabel, ylabel='ratio', title="$\dots$ all nodes")
        # a[j][0].set(xlabel=xAxisLabel, ylabel='Ratio of all nodes', title="$\dots$ all nodes")
    else:
        # a[j][0].set(xlabel=xAxisLabel, ylabel='Ratio of all nodes')
        a[j][0].set(xlabel=xAxisLabel, ylabel='ratio')
    # a0.legend(title='Algorithms')
    
    setLegendAlgoFont(a[j][0])
    
    sns.boxplot(data=dataE[(dataE['algoName']=='Feng-gs') & (dataE['skipped']==False)], x='graphName', y='exploredNodesRelYellow', hue='algoName',
                order=graphOrder, ax=a[j][1])
    if j == 0:
        a[j][1].set(xlabel=xAxisLabel, ylabel='', title="$\dots$ yellow nodes")
        # a[j][1].set(xlabel=xAxisLabel, ylabel='Ratio of yellow nodes', title="$\dots$ yellow nodes")
    else:
        a[j][1].set(xlabel=xAxisLabel, ylabel='')
        # a[j][1].set(xlabel=xAxisLabel, ylabel='Ratio of yellow nodes')
        
    a[j][1].set_yscale('log', base=2)
    if j > 0:
        a[j][0].get_legend().remove()
    a[j][1].get_legend().remove()

# plt.ylim(None, 1)
plt.suptitle("Nodes explored by $\Delta$-Stepping relative to $\dots$", fontsize=16)
finalizePlot(f'plots/early-stopping/explored-nodes-combined.pdf')
    
#     ax = sns.lineplot(data=dataE[(dataE['skipped']==False)], x='graph', y='exploredNodesRelTotal', hue='algoName', hue_order=algoOrder)
        
#     plt.yscale('log')
#     ax.set(xlabel=xAxisLabel, ylabel='Ratio of all nodes',
#            title="Ratio of nodes explored by $\Delta$-Stepping")
#     # plt.legend(title='Algorithms')
    
#     setLegendAlgoFont(ax)
    
#     finalizePlot(
#         None
#         # f'plots/early-stopping/explored-nodes-line-{graph}.pdf'
#     )
    

# Early Stopping

In [ ]:
fig, a = plt.subplots(1, 3, sharey=True, gridspec_kw={'width_ratios': [9, 7, 4]}, figsize=(10, 5))

for j, graph in enumerate(['gnp4','gnp64','grids']):
    if graph == 'gnp4':
        dataES = stoppedEarlyData4
        graphList = range(20, 29)
        xAxisLabel = '$n$ of $\mathcal{G}(n,\\frac{4}{n})$ graph'
    elif graph == 'gnp64':
        dataES = stoppedEarlyData64
        graphList = range(20, 27)
        xAxisLabel = '$n$ of $\mathcal{G}(n,\\frac{64}{n})$ graph'
    elif graph == 'grids':
        dataES = stoppedEarlyDataGrids
        graphList = range(18, 25, 2)
        xAxisLabel = '$n$ of $\operatorname{Grid}(n,4,0.8)$ graph'
    
    algoOrder = ['Feng-gs', 'Yen-gs', 'Yen-s']
    graphOrder = [f'$2^{{{i}}}$' for i in graphList]
    
    # ax = sns.boxplot(data=dataES, x='n', y='stopped-early-rel-total', hue='algoName', order=graphOrder, hue_order=algoOrder)
    # ax.set(xlabel=xAxisLabel, ylabel='ratio',
    #        title="Ratio of SSSP computations stopped without reaching the target")
    # setLegendAlgoFont(ax)
    # finalizePlot(f'plots/early-stopping/stopped-early-total-{graph}.pdf')
    
    sns.boxplot(data=dataES, x='n', y='stopped-early-rel-non-skipped', hue='algoName', order=graphOrder, hue_order=algoOrder, ax=a[j])
    a[j].set(xlabel=xAxisLabel, ylabel='ratio',
           #title="Ratio of SSSP computations stopped without reaching the target"
          )
    
a[1].set_ylabel('')
a[2].set_ylabel('')
plt.suptitle("Ratio of SSSP computations stopped without reaching the target", fontsize=16)
a[0].get_legend().remove()
a[2].get_legend().remove()
setLegendAlgoFont(a[1])
finalizePlot('plots/early-stopping/stopped-early-non-skipped-combined.pdf')
